# 🚀 AI Video Upscale & Interpolation Workflow

Welcome Huy! This notebook is designed to upscale your videos to 4K using **Real-ESRGAN** and optionally boost them to 60fps using **RIFE**.

### 📌 Workflow Overview:
1. **Connect Google Drive**
2. **Install Dependencies** (Real-ESRGAN & RIFE)
3. **Upscale to 4K** (Real-ESRGAN)
4. **Interpolate to 60fps** (RIFE)
5. **Finalize** (Merge Audio/Subs & Encode H.265 MKV)

## 1️⃣ Setup Environment
First, mount your Google Drive and set up the GPU.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Check GPU Status
!nvidia-smi

## 2️⃣ Install Tools
This installs Real-ESRGAN and RIFE repositories.

In [ ]:
# Install Real-ESRGAN
!git clone https://github.com/xinntao/Real-ESRGAN.git
%cd Real-ESRGAN
!pip install basicsr facexlib gfpgan
!pip install -r requirements.txt
!python setup.py develop

# Pre-download weights (Anime Video v3)
!wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.5.0/realesr-animevideov3.pth -P weights/

In [ ]:
# Install RIFE
%cd /content
!git clone https://github.com/hzwer/Practical-RIFE.git RIFE
%cd RIFE
!gdown --id 1wsQIhf9Y7v_z5V06rX8Nia62nswqLpS0
!7z e RIFE_trained_model_v4.7.zip

## 3️⃣ Run Upscale (Real-ESRGAN)
Adjust the input path. This step generates a **temporary video file** (no audio/metadata yet).

In [ ]:
import os

INPUT_PATH = "/content/drive/MyDrive/Movies/Input/phim_cua_huy.mp4" # @param {type:"string"}
TEMP_UPSCALE_PATH = "/content/input_upscaled.mp4"
SCALE = 2 # @param {type:"integer"}
MODEL_NAME = "realesr-animevideov3" # @param ["realesr-animevideov3", "RealESRGAN_x4plus_anime_6B"]

%cd /content/Real-ESRGAN
!python inference_realesrgan_video.py \
    -n {MODEL_NAME} \
    -i {INPUT_PATH} \
    -o {TEMP_UPSCALE_PATH} \
    -s {SCALE}

## 4️⃣ Run 60FPS Interpolation (RIFE) [Optional]
If you want 60fps, run this. If not, skip this cell.

In [ ]:
TEMP_60FPS_PATH = "/content/input_upscaled_60fps.mp4"
EXPONENT = 1 # @param {type:"integer"} 
# Exponent 1 creates 2x frames (24 -> 48 or 30 -> 60).

%cd /content/RIFE
!python3 inference_video.py \
    --video={TEMP_UPSCALE_PATH} \
    --output={TEMP_60FPS_PATH} \
    --exp={EXPONENT}

# Update the temp path for the next step
TEMP_UPSCALE_PATH = TEMP_60FPS_PATH

## 5️⃣ Finalize: Encode H.265 MKV & Merge Metadata
This step takes the video from AI and merges it with **original audio, subtitles, and chapters** from your source file.
It also encodes using **H.265 (HEVC)** for high quality and small size.

In [ ]:
FINAL_OUTPUT_PATH = "/content/drive/MyDrive/Movies/Output/phim_4K_Final.mkv" # @param {type:"string"}

# FFmpeg Command Explanation:
# -c:v libx265 -> Encode video with H.265 (High Efficiency)
# -crf 23 -> Optimal 4K quality/size balance (Standard for 4K is 22-24)
# -preset fast -> Faster encoding speed (Good balance for Colab CPU)
# -c:a copy -> Copy audio exactly
# -c:s copy -> Copy subtitles exactly
# -map_metadata 0 -> Keep all original details

!ffmpeg -i {INPUT_PATH} -i {TEMP_UPSCALE_PATH} \
    -map 1:v -map 0:a? -map 0:s? \
    -c:v libx265 -tag:v hvc1 -crf 23 -preset fast \
    -c:a copy -c:s copy \
    -map_metadata 0 \
    {FINAL_OUTPUT_PATH}